### Importing libraries

In [88]:
import sys
sys.path.append("../")
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot


In [89]:
from infrastructure.instrument_collection import instrumentCollection as ic

### Reading data

In [90]:
pair = "EUR_USD"
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20, 50, 100, 200]

In [91]:
df.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [92]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [93]:
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
    
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)

In [94]:
df_plot = df_ma.iloc[:100]

In [95]:
cp = CandlePlot(df_plot)

In [96]:
traces = [f'MA_{x}' for x in MA_LIST]

In [97]:
cp.show_plot(line_traces=traces)

In [98]:
MA_S = 'MA_50'
MA_L = 'MA_200'
BUY = 1 
SELL = -1
NONE = 0

In [99]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [100]:
df_an.head(20)

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2020-09-17 05:00:00+00:00,1.17590,1.18122,1.17556,1.18038,1.182965,1.183533
1,2020-09-17 09:00:00+00:00,1.18036,1.18128,1.17832,1.18082,1.182914,1.183549
2,2020-09-17 13:00:00+00:00,1.18082,1.18278,1.17938,1.18222,1.182893,1.183571
3,2020-09-17 17:00:00+00:00,1.18221,1.18525,1.18212,1.18478,1.182943,1.183641
4,2020-09-17 21:00:00+00:00,1.18478,1.18621,1.18440,1.18540,1.183019,1.183693
5,2020-09-18 01:00:00+00:00,1.18538,1.18615,1.18432,1.18488,1.183089,1.183736
6,2020-09-18 05:00:00+00:00,1.18487,1.18691,1.18400,1.18592,1.183193,1.183789
7,2020-09-18 09:00:00+00:00,1.18591,1.18638,1.18306,1.18444,1.183259,1.183824
8,2020-09-18 13:00:00+00:00,1.18445,1.18704,1.18262,1.18656,1.183374,1.183861
9,2020-09-18 17:00:00+00:00,1.18655,1.18675,1.18387,1.18400,1.183452,1.183917


In [102]:
df_an["DELTA"] = df_an.MA_50 - df_an.MA_200
df_an["DELTA_PREV"] = df_an.DELTA.shift(1)

In [103]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    else:
        return NONE

In [104]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [105]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [106]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
145,2020-10-21 09:00:00+00:00,1.18506,1.18662,1.18415,1.18628,1.177146,1.177008,0.000137,-0.000032,1
200,2020-11-03 14:00:00+00:00,1.17046,1.17399,1.17024,1.17096,1.174794,1.175009,-0.000216,0.000007,-1
236,2020-11-11 14:00:00+00:00,1.17585,1.17744,1.17456,1.17730,1.176056,1.175962,0.000094,-0.000056,1
514,2021-01-18 22:00:00+00:00,1.20768,1.20970,1.20740,1.20934,1.217530,1.217906,-0.000376,0.000105,-1
671,2021-02-24 02:00:00+00:00,1.21588,1.21624,1.21456,1.21481,1.211605,1.211576,0.000029,-0.000068,1


In [132]:
cp = CandlePlot(df_an.iloc[230:514])
cp.show_plot(line_traces=[MA_S, MA_L])

In [108]:
ic.LoadInstruments("../data")

In [109]:
ic.instruments_dict[pair]

{'name': 'EUR_USD', 'ins_type': 'CURRENCY', 'displayName': 'EUR/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.0333}

In [110]:
ins_data = ic.instruments_dict[pair]

In [118]:
df_trades.shape

(22, 13)

In [119]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE,DIFF,GAIN,GAIN_C
145,2020-10-21 09:00:00+00:00,1.18506,1.18662,1.18415,1.18628,1.177146,1.177008,0.000137,-0.000032,1,-0.01532,-153.2,-153.2
200,2020-11-03 14:00:00+00:00,1.17046,1.17399,1.17024,1.17096,1.174794,1.175009,-0.000216,0.000007,-1,0.00634,-63.4,-216.6
236,2020-11-11 14:00:00+00:00,1.17585,1.17744,1.17456,1.17730,1.176056,1.175962,0.000094,-0.000056,1,0.03204,320.4,103.8
514,2021-01-18 22:00:00+00:00,1.20768,1.20970,1.20740,1.20934,1.217530,1.217906,-0.000376,0.000105,-1,0.00547,-54.7,49.1
671,2021-02-24 02:00:00+00:00,1.21588,1.21624,1.21456,1.21481,1.211605,1.211576,0.000029,-0.000068,1,-0.01885,-188.5,-139.4


In [120]:
df_trades["DIFF"] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [121]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE'] 

In [126]:
df_trades.GAIN.sum()

26.399999999999864

In [127]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [128]:
cp = CandlePlot(df_trades, candles=False)

In [129]:
cp.show_plot(line_traces=['GAIN_C'])